In [398]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb
import string


def print_mat(mat):
    for row in mat:
        print(''.join(row))


files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [403]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

y = 10 if box.value == "ex1.txt" else 2_000_000

pat = re.compile('x=(-?\d+), y=(-?\d+)')
S = np.matrix([sum(list(map(lambda t: list(map(int, t)), pat.findall(line))), [])
     for line in lines])

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [404]:
def manhattan(m1, m2):
    return np.squeeze(np.asarray(np.sum(np.abs(m1 - m2), axis=1).reshape(-1,)))


DS = manhattan(S[:, :2], S[:, 2:])

padding = np.matrix([DS, np.zeros(len(DS), dtype=int)]).T
minx, _ = np.amin(S[:, :2] - padding, axis=0).tolist()[0]
maxx, _ = np.amax(S[:, :2] + padding, axis=0).tolist()[0]


Y = np.matrix([np.arange(minx, maxx, dtype=int).repeat(len(S)),
               np.full(maxx-minx, y, dtype=int).repeat(len(S))]).T
DY = np.reshape(
    manhattan(np.tile(S[:, :2], (maxx-minx, 1)), Y), (maxx-minx, len(S)))

DSY = DY - np.tile(DS, (maxx-minx, 1))
DSY[DSY <= 0] = -1
DSY[DSY > 0] = 0

DSY = np.sum(DSY, axis=1)

b = len(set([tuple(row) for row in S[:, 2:].tolist() if row[1] == y]))

pprint(len(DSY[DSY != 0]) - b)


4560025


### Part 2

In [405]:
maxy = y * 2

done = False
for i in range(len(S)):
    if done:
        break

    x, y = S[i, :2].tolist()[0]
    ds = DS[i]
    I = np.arange(len(S))
    r = np.arange(0, ds+2)
    for dx, dy in [(1, 1), (-1, 1), (1, -1), (-1, -1)]:
        D = np.matrix([r*dx, r[::-1]*dy]).T
        P = np.matrix([x, y]) + D
        M = np.reshape(manhattan(np.tile(P, (len(S) - 1, 1)),
                       S[I != i, :2].repeat(ds + 2, axis=0)), (len(S) - 1, ds+2))
        N = M.T - DS[I != i]
        N[N <= 0] = -1
        N[N > 0] = 0
        N = np.sum(N, axis=1)

        found = [p for p in P[N == 0, :].tolist()
                 if p[0] in range(0, maxy+1) and
                 p[1] in range(0, maxy+1)]
        if len(found) == 1:
            print(found[0][0] * 4_000_000 + found[0][1])
            done = True
            break


12480406634249
